# Detecting truck traffic in Sentinel-2 imagery

In [ ]:
import os

os.environ["VECTOR_API_HOST"] = "http://127.0.0.1:8000"

In [ ]:
from affine import Affine
import descarteslabs as dl
import geopandas as gpd
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import pyproj
from rasterio.features import shapes
import shapely.geometry
import shapely.ops

from descarteslabs.vector import Table, models

## Mosaic some Sentinel-2 imagery

Collect Sentinel-2 L2A imagery from which trucks will be extracted.

In [ ]:
lat, lon = 51.893789, 4.428885
dltile = dl.scenes.DLTile.from_latlon(lat, lon, 10, 4096, 0)

In [ ]:
scenes, ctx = dl.scenes.search(
    dltile,
    "esa:sentinel-2:l2a:v1",
    start_datetime="2020-04-19",
    end_datetime="2020-04-21",
)

In [ ]:
arr, raster_info = scenes.mosaic(
    ["blue", "green", "red", "nir", "swir1"], ctx, raster_info=True
)
B02, B03, B04, B08, B11 = arr

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(B03, vmin=0, vmax=0.4, cmap="gray")

## Detect vehicles in the imagery

Extract trucks from Sentinel-2 L2A imagery using band ratios and thresholds.

In [ ]:
min_rgb = 0.04
max_red = 0.15
max_green = 0.15
max_blue = 0.4
max_ndvi = 0.7  # quite high to account for mixed pixels
max_ndwi = 0.001
max_ndsi = 0.0001
min_b11 = 0.05
max_b11 = 0.55
min_green_ratio = 0.05
min_red_ratio = 0.1

In [ ]:
# Compute a roads mask using band ratios and thresholds
ndvi_mask = ((B08 - B04) / (B08 + B04)) < max_ndvi
ndwi_mask = ((B02 - B11) / (B02 + B11)) < max_ndwi
ndsi_mask = ((B03 - B11) / (B03 + B11)) < max_ndsi
low_rgb_mask = (B02 > min_rgb) * (B03 > min_rgb) * (B04 > min_rgb)
high_rgb_mask = (B02 < max_blue) * (B03 < max_green) * (B04 < max_red)
b11_mask = ((B11 - B03) / (B11 + B03)) < max_b11
b11_mask_abs = (B11 > min_b11) * (B11 < max_b11)
roads_mask = (
    ndvi_mask
    * ndwi_mask
    * ndsi_mask
    * low_rgb_mask
    * high_rgb_mask
    * b11_mask
    * b11_mask_abs
)

In [ ]:
bg_ratio = (B02 - B03) / (B02 + B03)
br_ratio = (B02 - B04) / (B02 + B04)

bg_low = (bg_ratio * roads_mask) > min_green_ratio
br_low = (br_ratio * roads_mask) > min_red_ratio
vehicles = bg_low * br_low

## Vectorize the detections

 Convert the detected trucks to vector format.

In [ ]:
transform = Affine.from_gdal(*raster_info["geoTransform"])

In [ ]:
vehicles_vector = list(shapes(vehicles.data.astype(np.uint8), transform=transform))

In [ ]:
utm_points = [
    shapely.geometry.shape(vehicle[0]).centroid for vehicle in vehicles_vector
]

In [ ]:
epsg_code = raster_info["coordinateSystem"]["epsg"]

wgs84 = pyproj.CRS("EPSG:4326")
utm = pyproj.CRS(f"EPSG:{epsg_code}")

transform = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform
wgs84_points = [shapely.ops.transform(transform, point) for point in utm_points]

In [ ]:
vehicles = gpd.GeoDataFrame(data={"geometry": wgs84_points}, crs="EPSG:4326")
vehicles.head()

## Ingest the detections into a Vector product

Before creating the Vector product, ensure it does not already exist.

In [ ]:
orgname = dl.auth.Auth().payload["org"]

for table in Table.list():
    if table.id == f"{orgname}:trucks":
        print(f'Deleting "{table}"')
        table.delete()

Vector allows for creation of custom schemas for each Vector product. The geometry and UUID columns are inherited from the parent model, `models.PointBaseModel`, and additional indices can be specified using pydantic Fields. In this example, the detections only have a geometry attribute, thus the model `models.PointBaseModel` is being used without alteration. Creating the product will return a `Table` object.

In [ ]:
truck_table = Table.create(
    "trucks", "Trucks", owners=["org:descarteslabs"], model=models.PointBaseModel
)

Features can be uploaded/ingested to the Vector product by invoking the method `Table.add()`. Adding features will return a `GeoPandas.GeoDataFrame` with UUID attribution.

In [ ]:
gdf = truck_table.add(vehicles)

In [ ]:
gdf.head()

## Visualize the detections

Vector products can be visualized by calling the `Table.visualize()` method which will return a vector tile layer compatible with ipyleaflet. Vector visualization also supports the use of `TableOptions`; however, only the property filter and columns will be honored.

In [ ]:
m = ipyleaflet.Map(
    center=(lat, lon),
    zoom=10,
    scroll_wheel_zoom=True,
)
m

In [ ]:
# add a layer style
vector_tile_layer_styles = {
    "fill": "true",
    "fillColor": "#ff0000",
    "color": "#000000",
    "weight": 1,
    "fillOpacity": 1,
    "radius": 3,
}

lyr = truck_table.visualize(
    name="trucks", map=m, vector_tile_layer_styles=vector_tile_layer_styles
)

## Deleting a Vector product

To delete a Vector product, simply invoke the `Table.delete()` method.

In [ ]:
truck_table.delete()